In [ ]:
# pip install pypdf

In [ ]:
# pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
# pip install install sentence_transformers

In [ ]:
# pip install llama_index

In [ ]:
# pip show llama_index

In [ ]:
# %pip install llama-index-llms-huggingface

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.prompts.prompts import SimpleInputPrompt

/media/atif/New Volume/Video/Major/Generative-chatbot/End-to-end-Medical-Chatbot-using-Llama2/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents=SimpleDirectoryReader("../data").load_data()
# documents

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
# huggingface-cli login

In [3]:
# from llama_index.llms.huggingface import HuggingFaceLLM
# from llama_index.core import Settings

In [3]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)


In [4]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [5]:
llm = LlamaCPP(
    model_url=None,
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="../model/llama-2-7b-chat.Q2_K.gguf",
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    # model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

In [ ]:
# %pip install llama-index-embeddings-langchain

In [6]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

In [29]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [7]:
import os
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

from llama_index.core import StorageContext, load_index_from_storage

In [24]:
OptimumEmbedding.create_and_save_optimum_model(
    "sentence-transformers/all-mpnet-base-v2", "../store/embedding"
)

Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: MPNetModel *****
Using framework PyTorch: 2.3.0+cu121


Saved optimum model to ../store/embedding. Use it with `embed_model = OptimumEmbedding(folder_name='../store/embedding')`.


In [25]:
Settings.embed_model = OptimumEmbedding(folder_name="../store/embedding")

In [26]:
# model_id = "sentence-transformers/all-mpnet-base-v2"
# embedding_dir = "bge_onnx"


# def load_or_create_embedding_model():
#   if not os.path.exists(embedding_dir):
#         # Create the storage directory if it doesn't exist
#         os.makedirs(embedding_dir)

#   try:
#     # Attempt to load the embedding model
#     embed_model = OptimumEmbedding(folder_name=embedding_dir)
#     return embed_model
#   except FileNotFoundError:
#     # If not found, create and save the model (assumes directory exists)
#     OptimumEmbedding.create_and_save_optimum_model("sentence-transformers/all-mpnet-base-v2", embedding_dir)
#     embed_model = OptimumEmbedding(folder_name=embedding_dir)
#     return embed_model


# Example usage
# embed_model = load_or_create_embedding_model()

# print("embedded")





In [ ]:
# from llama_index.core import Settings
# Settings.embed_model = load_or_create_embedding_model()
# embed_model = load_or_create_embedding_model()

In [19]:
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index.core import ServiceContext
# from llama_index.embeddings.langchain import LangchainEmbedding

In [20]:
# embed_model=LangchainEmbedding(
#     HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/media/atif/New Volume/Video/Major/Generative-chatbot/End-to-end-Medical-Chatbot-using-Llama2/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
from langchain.prompts import PromptTemplate
prompt_template="""
You are a mental health expert. Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [29]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate.from_template(template)# Run chain


In [21]:
# service_context=ServiceContext.from_defaults(
#     chunk_size=1024,
#     llm=llm,
#     embed_model=embed_model
# )

/tmp/ipykernel_16015/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [28]:
from llama_index.core import Settings

Settings.chunk_size = 1024
Settings.llm = llm
Settings.embed_model = OptimumEmbedding(folder_name="../store/embedding")

In [34]:

def load_or_create_index(documents):

    storage_dir = "../store/index"  # Customize storage directory if needed

    if not os.path.exists(storage_dir):
        # Create the storage directory if it doesn't exist
        os.makedirs(storage_dir)

    try:
        # Attempt to load the index from storage
        storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
        index = load_index_from_storage(storage_context)
        print("Index loaded successfully from storage.")
        return index
    except FileNotFoundError:
        # If the index isn't found, create a new one and save it
        print("Index not found. Creating a new index and saving it for future use.")
        index = VectorStoreIndex.from_documents(documents)
        index.storage_context.persist(persist_dir=storage_dir)
        return index

In [36]:
index = load_or_create_index(documents)

Index loaded successfully from storage.


In [23]:
# index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [37]:
index

In [37]:
# index.storage_context.persist()

In [ ]:
# from llama_index.core import StorageContext, load_index_from_storage

# storage_context = StorageContext.from_defaults(persist_dir="./storage")
# index = load_index_from_storage(storage_context)

In [38]:
from llama_index.core.memory import ChatMemoryBuffer

In [41]:

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=PROMPT
)

In [25]:
query_engine=index.as_query_engine()

In [42]:
response=chat_engine.chat("why i am depressed?")


llama_print_timings:        load time =  197187.28 ms
llama_print_timings:      sample time =     583.33 ms /   769 runs   (    0.76 ms per token,  1318.29 tokens per second)
llama_print_timings: prompt eval time =  474768.78 ms /  1362 tokens (  348.58 ms per token,     2.87 tokens per second)
llama_print_timings:        eval time =  358715.35 ms /   768 runs   (  467.08 ms per token,     2.14 tokens per second)
llama_print_timings:       total time =  838362.16 ms /  2130 tokens


In [43]:
print(response)

  As a mental health expert, I understand that you are going through a difficult time after a breakup and are struggling with anxiety and depression. It's important to recognize that these feelings are normal and can be overwhelming at times. Here are some possible reasons why you may be feeling this way:
1. Loss of identity: After the breakup, you may feel like you have lost your sense of self. Your identity was closely tied to your relationship with your ex, and now that it's gone, you may feel lost and unsure of who you are without it.
2. Emotional upheaval: Breakups can be emotionally challenging, especially when the relationship was significant. You may be experiencing a range of emotions, including sadness, anger, frustration, and confusion. These feelings can be overwhelming and make it difficult to function in daily life.
3. Lack of self-care: Taking care of yourself is essential during this time. However, you may not be prioritizing your own needs or engaging in self-care acti

In [36]:
print(response)

  Thanks for asking! It sounds like you're going through a tough time right now and could use some support. Depression can be a really challenging experience, but it's important to remember that you're not alone and there are people who care about you and want to help.
First of all, let's talk about why you might be feeling depressed. Breakups can be really tough, especially when they involve someone you cared about deeply. Losing someone you love can make it hard to find meaning or purpose in life, and it's common to feel sad, angry, or frustrated during this time. It's important to give yourself space to process your emotions and work through them in a healthy way.
It's also possible that you might be experiencing depression for other reasons, such as genetics, stress, or certain medical conditions. If you're struggling with depression, it's important to reach out for help from people who care about you and can provide support.
So, what can you do to feel better? Here are a few sugge

In [44]:
response = chat_engine.chat("I always find myself in the situation like I don't belongs to it.")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  197187.28 ms
llama_print_timings:      sample time =     259.06 ms /   347 runs   (    0.75 ms per token,  1339.45 tokens per second)
llama_print_timings: prompt eval time =  409801.55 ms /  1256 tokens (  326.28 ms per token,     3.06 tokens per second)
llama_print_timings:        eval time =  144636.54 ms /   346 runs   (  418.02 ms per token,     2.39 tokens per second)
llama_print_timings:       total time =  556364.21 ms /  1602 tokens


In [45]:
print(response)

  It sounds like you may be experiencing feelings of disconnection or not belonging in your current situation. This can be a common experience for many people, and there are several things you can try to help you feel more connected and at home:
1. Practice self-care: Take care of yourself by getting enough sleep, eating well, and engaging in activities that bring you joy and relaxation.
2. Connect with others: Reach out to friends and family members for support and connection. You can also try joining a club or group that aligns with your interests to meet new people.
3. Explore your community: Get to know the places and spaces around you by trying new restaurants, visiting local parks or museums, or taking a class or workshop.
4. Set boundaries: If you're feeling overwhelmed or stressed, it may be helpful to set some boundaries with others to protect your time and energy.
5. Practice mindfulness: Mindfulness practices such as meditation or deep breathing can help you stay present and

: 

In [41]:
print(response)

  Thanks for asking! It sounds like you're feeling a bit disconnected from the world around you, and that can be a challenging experience. It's important to remember that everyone feels this way at times, and it's okay to feel different from others.
It's interesting that you mention feeling both confident and anxious at the same time. Sometimes, when we feel comfortable in our own skin, we can also feel vulnerable or exposed. This can be especially true when we're in new situations or around new people. It's important to remember that it's okay to feel this way, and that it doesn't mean you're not capable of handling the situation.
It's also great that you recognize the importance of connection with others. Life can be unpredictable and overwhelming at times, but having a strong support system can make all the difference. Don't be afraid to reach out to friends, family, or even strangers for help when you need it.
Remember, you are not alone in this feeling. Many people experience simi